In [9]:
import requests
from bs4 import BeautifulSoup
import html5lib
from pprint import pprint
import pandas as pd
import numpy as np
import re

In [2]:
headers = {'user-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13B137 Safari/601.1'}
url = 'https://spb.hh.ru/search/vacancy' #У меня браузер открывает только так, с spb


In [11]:
def find_vacations(vac_name):
    ### Понимаю, что код с двумя вложенными циклами трудно назвать оптимальным, но на что-то лучшее, к сожалению, не хватило времени. Очень большие выдачи считает
        ### довольно долго (5-7 мин)
    ### Максимальное число ответов в выдаче, независимо от их реального числа, - не более 2000. Видимо, опять сайт режет?
    ### Добавил в выдачу еще и работодателя, хотя этого не было в задании.
    params = {'text' : vac_name}
    response = requests.get(url, headers=headers, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    c = soup.find('h1', {'data-qa': 'bloko-header-3'})
    x = re.findall('[0-9]+', c.get_text())
    number = int(''.join(map(str, x)))
    number = int(number/20) #Нашел на странице строчку, где указано общее число запросов в выдаче, вынул оттуда это число, разделил на 20 (это значение числа вакансий #на странице установил принудительно ниже) и так нашел общее число страниц в выдаче

    lst = []
    page  = 0
    while page <= number:
        params = {'text' : vac_name, 'page' : page, 'items_on_page' : 20}
        response = requests.get(url, headers=headers, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')
        vacations = soup.find_all('div', {'class': 'vacancy-serp-item'})
        for vac in vacations:
            temp = {}
            a=vac.find_all('span', {'class': 'g-user-content'})
            for elt in a:
                temp['Должность'] = elt.text
            b=vac.find_all('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            for elt in b:
                temp['Зарплата'] = elt.text.replace('\u202f','')
            c=vac.find_all('a', {'data-qa': 'vacancy-serp__vacancy-employer'}) 
            for elt in c:
                temp['Работодатель'] = elt.text
            d=vac.find_all('a', {'data-qa': 'vacancy-serp__vacancy-title'}, href=True) 
            for elt in d:
                temp['Ссылка на вакансию'] = elt['href']  
            lst.append(temp)
        page += 1
    x = pd.DataFrame(lst)

    x['Валюта'] = x['Зарплата'].replace('[0-9–дот]','',regex=True)
    x['Мин.зарплата'] = np.select([x['Зарплата'].str.contains('от',na=False), x['Зарплата'].str.contains('до',na=False), x['Зарплата'].str.contains('–',na=False)],
                                [x['Зарплата'].replace('[^\d]','',regex=True),0, x['Зарплата'].replace('–.*','',regex=True)])
    
    x['Макс.зарплата'] = np.select([x['Зарплата'].str.contains('от',na=False), x['Зарплата'].str.contains('до',na=False), x['Зарплата'].str.contains('–',na=False)],
                                [0, x['Зарплата'].replace('[^\d]','',regex=True), x['Зарплата'].replace('.*–','',regex=True).replace('[^\d]','',regex=True)])
    x.fillna('не указана', inplace=True)
    x.drop('Зарплата',axis=1,inplace=True)
    x = x.reindex(columns=['Должность', 'Работодатель', 'Мин.зарплата', 'Макс.зарплата', 'Валюта', 'Ссылка на вакансию'])
    x[['Мин.зарплата', 'Макс.зарплата']] = x[['Мин.зарплата', 'Макс.зарплата']].astype('int')
    x.index +=1

    display(x)

In [12]:
find_vacations('middle data scientist')

,Должность,Работодатель,Мин.зарплата,Макс.зарплата,Валюта,Ссылка на вакансию
1,Data Scientist в области рекомендательных систем,Сбер. IT,0,0,не указана,https://spb.hh.ru/vacancy/54682944?from=vacanc...
2,Data scientist (Junior/Middle) в консалтинг,ООО БСТ Менеджмент-Консалтинг,0,0,не указана,https://spb.hh.ru/vacancy/55074464?from=vacanc...
3,Middle Data Scientist,ООО Телега,200000,250000,руб.,https://spb.hh.ru/vacancy/54955066?from=vacanc...
4,"Математик, Junior/Middle/Senior Data Scientist",Представительство ООО «Квалитет» (Российская Ф...,0,0,не указана,https://spb.hh.ru/vacancy/55529384?from=vacanc...
5,Data Scientist (Marketplace Efficiency),inDriver,0,0,не указана,https://spb.hh.ru/vacancy/55485778?from=vacanc...
6,Data Scientist (Middle),билайн: ИТ и Digital,0,0,не указана,https://spb.hh.ru/vacancy/50630870?from=vacanc...
7,Senior Data Scientist (computer vision),inDriver,0,0,не указана,https://spb.hh.ru/vacancy/54717873?from=vacanc...
8,Senior Data Scientist (Antifraud),inDriver,0,0,не указана,https://spb.hh.ru/vacancy/54789914?from=vacanc...
9,Data scientist / Machine learning engineer (ко...,VK,0,0,не указана,https://spb.hh.ru/vacancy/50605612?from=vacanc...
10,Middle Data Scientist,Click,0,260000,руб.,https://spb.hh.ru/vacancy/53014416?from=vacanc...
